In [ ]:
import matplotlib.pyplot as plt
import os
import re
import string
import tensorflow as tf
import pandas as pd

from utils import predicted_test_data_to_result_csv
from keras import layers, losses, Input, Model
from keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, BatchNormalization, Activation, Flatten
from keras.losses import sparse_categorical_crossentropy
from keras.metrics import sparse_categorical_accuracy
from keras.optimizers import Adam, SGD

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [ ]:
train_path = "data/base/goodreads_train.csv"
result_path = "data/base/goodreads_test.csv"

In [ ]:
df = pd.read_csv(train_path, sep=",")

Drop all lines with 0 in rating column

In [ ]:
index = df[(df['rating'] == 0)].index
df.drop(index, inplace=True)
df.reset_index(inplace=True, drop=True)

Targets DataFrames

In [ ]:
targets = df.pop('rating')
targets = targets - 1
# targets = tf.keras.utils.to_categorical(targets)

Features DataFrames

In [ ]:
features_names = ['review_text']
features = df[features_names]
tf.convert_to_tensor(features)

In [ ]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_spoilers = tf.strings.regex_replace(lowercase, '\*\* spoiler alert \*\*', ' ')
    return tf.strings.regex_replace(stripped_spoilers,
                                    '[%s]' % re.escape(string.punctuation),
                                    '')

In [ ]:
max_features = 5000  # Maximum vocab size.
sequence_length = 100

In [ ]:
vectorized_layer = tf.keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [ ]:
vectorized_layer.adapt(features)

In [ ]:
epochs = 50
model_nb = 1

embedding_dim = 50
learning_rate = 0.01
batch_size = 1024
dropout_rate = 0.0

In [ ]:
input_text = Input(shape=(1,), dtype=tf.string)

vectorized_text = vectorized_layer(input_text)

embedding_layer = Embedding(max_features + 1, embedding_dim, input_length=sequence_length)(vectorized_text)

x_shortcut = embedding_layer

#### Main path ####
# First
x = Conv1D(64, 3, padding = 'valid')(embedding_layer)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# Second
x = Conv1D(128, 5, padding = 'valid')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# Third
x = Conv1D(256, 5, padding = 'valid')(x)
x = BatchNormalization()(x)

#### Shortcut path ####
x_shortcut = Conv1D(256, 11, padding = 'valid')(x_shortcut)
x_shortcut = BatchNormalization()(x_shortcut)

# x and x_shortcut addition
x = x + x_shortcut

global_max_pooling = GlobalMaxPooling1D()(x)

relu = Dense(32, activation='relu')(global_max_pooling)

output = Dense(5, activation='softmax')(relu)

resnet_model = Model(input_text, output)

resnet_model.summary()

In [ ]:
resnet_model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate=learning_rate),
                  metrics=sparse_categorical_accuracy)

In [ ]:
exp_name = f'rnn_model_{model_nb}_lr_{learning_rate}_bs_{batch_size}_dr_{dropout_rate}'

In [ ]:
resnet_model.fit(features,
              targets,
              validation_split=0.25,
              batch_size=batch_size,
              epochs=epochs,
              callbacks=[tf.keras.callbacks.TensorBoard("logs/rnn/" + exp_name)])

In [ ]:
df_test = pd.read_csv(result_path, sep=",")

df_test_modified = df_test.drop(columns=[
    'user_id',
    'book_id',
    'review_id',
    'date_added',
    'date_updated',
    'read_at',
    'started_at',
    'n_votes',
    'n_comments'
], inplace=False)

In [ ]:
predicted_test_data = resnet_model.predict(df_test_modified)

In [ ]:
df_test.head()

In [ ]:
predicted_test_data_to_result_csv(df_test, predicted_test_data, exp_name)